In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import scipy.io
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.ticker import FormatStrFormatter

from tueplots import figsizes
from sbi_ice.utils import plotting_utils,misc
data_dir,output_dir = misc.get_data_output_dirs()
color_opts = plotting_utils.setup_plots()


In [ ]:
print(Path(data_dir,"Ekstrom","IRH_data","Ekstroem_flowline_GPR_IRH_data.mat"))
mat = scipy.io.loadmat(Path(data_dir,"Ekstrom","IRH_data","Ekstroem_flowline_GPR_IRH_data.mat"))
shelf_mask = scipy.io.loadmat(Path(data_dir,"Ekstrom","IRH_data","LayerData_flowline_mask.mat"))['save_in'].astype(bool).flatten()
print(mat.keys())
print(mat["data"].shape)

In [ ]:
distance = mat['distance'].flatten()
print(distance.shape)
indices = np.arange(0,distance.size)
print(mat["elev_layers"].shape)
nl,ns = mat["elev_layers"].shape

base = mat["elev_sur_bedmachine"].flatten() - mat["thickness_bedmachine"].flatten()

xg = -296432
yg=1979538
xg1 = -291673
yg1=1996013
gl_pos = distance[shelf_mask][0]
minind = np.argmin(((mat["psX"][0]-xg)**2+(mat["psY"][0]-yg)**2))
minind1 = np.argmin(((mat["psX"][0]-xg1)**2+(mat["psY"][0]-yg1)**2))

RefHeight = np.max(mat["elev_sur_bedmachine"].flatten()[shelf_mask]+0.1*np.max(mat["elev_sur_bedmachine"].flatten()))
print(RefHeight)
MinElevation = -1200

ylim_base=[-150,100]
crit_indices = [2715,3115]
mask = (indices > crit_indices[0]) & (indices < crit_indices[1])
xlim_base = [distance[crit_indices[0]],distance[crit_indices[1]]]


In [ ]:
# Calculate elevations of radar data for radargram overlay

from sbi_ice.utils import plotting_utils,misc,modelling_utils,noise_model

eps_i = 3.15
eps = (np.sqrt(3)*eps_i)**3
c_light = 3.0e8
c_ice = c_light/np.sqrt(eps)
z = np.linspace(0.0,1000.0,10000)
rho = 917.0
def ref_ind_const_density(rho):
    return ((1/noise_model.rho_ice)*rho*(np.cbrt(3.15)-1)+1)**3
def twt2depth_const_density(twt,z,rho):
    print(twt.shape)
    v = c_light/(np.sqrt(ref_ind_const_density(rho)))

    dts = np.concatenate((np.array([0]),np.diff(z)))/v
    traveltimedepth = np.cumsum(dts)
    depth = np.zeros_like(twt)
    for i in range(0,twt.shape[0]):
        depth[i] = np.nan if not twt[i]>0 else z[np.argmin(np.abs(traveltimedepth-twt[i]/2))]
    return depth


depth = twt2depth_const_density(mat["twt"].flatten()[1:]/1e9,z,rho)
elevations = (mat["elev_sur_bedmachine"].T - depth).T
distances = np.tile(distance,(elevations.shape[0],1))

# norm_elevations =

In [ ]:
def radarplot(x,show_radargram=False,xmask = None,box = True,xlim=None,ylim=None,label_layers = False):
    plt.rcParams.update(figsizes.icml2022_full(height_to_width_ratio=0.7))


    if xmask is None:
        xmask = np.ones(x.shape,dtype=bool)
    if show_radargram:
        plt.rcParams.update(figsizes.icml2022_full(height_to_width_ratio=1.0))
        plt.rcParams.update({'font.size': 12,"axes.titlesize": 12,"axes.labelsize": 12,"xtick.labelsize" : 12,"ytick.labelsize": 12})
        fig,ax = plt.subplots()
        ax.pcolormesh(distances[:,shelf_mask*mask]-gl_pos,elevations[:,shelf_mask*mask],mat["data"][1:,shelf_mask*mask],cmap="seismic")
    else:
        plt.rcParams.update(figsizes.icml2022_full(height_to_width_ratio=0.7))
        plt.rcParams.update({'font.size': 12,"axes.titlesize": 12,"axes.labelsize": 12,"xtick.labelsize" : 12,"ytick.labelsize": 12})
        fig,ax = plt.subplots()
        ax.fill_between(x[xmask]-gl_pos,mat["elev_sur_bedmachine"].flatten()[xmask],base[xmask],color="black",alpha=0.075,linewidth=0.0)

    if box:
        rect = mpl.patches.Rectangle((xlim_base[0]-gl_pos, ylim_base[0]), xlim_base[1]-xlim_base[0], ylim_base[1]-ylim_base[0],
                                        linestyle="solid", linewidth=1.5, edgecolor=color_opts["colors"]["boundary_condition"], facecolor="none",
                                        zorder=10)
        ax.add_patch(rect)
    else:
        ax.set_xticks(np.linspace(x[xmask][0]-gl_pos,x[xmask][-1]-gl_pos,5))
    for kk in range(0,nl):
        ax.plot(x[xmask]-gl_pos,mat["elev_layers"][kk,xmask],color = color_opts["colors"]["observation"],linewidth=1.0)
        if label_layers:
            ax.text(x[xmask][kk*10]-gl_pos,mat["elev_layers"][kk,xmask][kk*10],str(kk+1),fontsize=15,color="black",
                    horizontalalignment='left',verticalalignment='center')
    ax.plot(x[xmask]-gl_pos,base[xmask],"black",linewidth=1.0)
    ax.plot(x[xmask]-gl_pos,mat["elev_bed_bedmachine"].flatten()[xmask],linewidth=1.0,color="black")
    ax.plot(x[xmask]-gl_pos,mat["elev_sur_bedmachine"].flatten()[xmask],linewidth=1.0,color="black")




    coords = np.vstack((np.concatenate((x[xmask]-gl_pos, x[xmask][::-1]-gl_pos)), np.concatenate((mat["elev_sur_bedmachine"].flatten()[xmask]*0+RefHeight, mat["elev_sur_bedmachine"].flatten()[xmask][::-1]))))
    patch1 = mpl.patches.Polygon(xy=coords.T,color=np.array([136, 233, 253])/255) #çolor in sky
    coords = np.vstack((np.concatenate((x[xmask]-gl_pos, x[xmask][::-1]-gl_pos)), np.concatenate((base[xmask], base[xmask][::-1]*0 + MinElevation))))
    patch2 = mpl.patches.Polygon(xy=coords.T,color=np.array([119, 158, 185])/255) #color in ocean
    coords = np.vstack((np.concatenate((x[xmask]-gl_pos, x[xmask][::-1]-gl_pos)), np.concatenate((mat["elev_bed_bedmachine"].flatten()[xmask], mat["elev_bed_bedmachine"].flatten()[xmask][::-1]*0 + MinElevation)))) #color in ground
    patch3 = mpl.patches.Polygon(xy=coords.T,color=np.array([168, 104, 66])/255)
    ax.add_patch(patch1)
    ax.add_patch(patch2)
    ax.add_patch(patch3)
    if xlim is not None:
        ax.set_xlim(xlim)
    if ylim is not None:
        ax.set_ylim(ylim)

    ax.set_xlabel("Distance from GL [km]")
    ax.set_ylabel("Elevation [m.a.s.l.]")
    ax.spines['bottom'].set_bounds(x[xmask][0]-gl_pos+0.005,x[xmask][-1]-gl_pos)
    ax.xaxis.set_major_formatter(FormatStrFormatter('%.1f'))

    return fig,ax
# def radarplot(x,show_radargram=False,xmask = None,box = True,xlim=None,ylim=None,label_layers = False):
#     plt.rcParams.update(figsizes.icml2022_full(height_to_width_ratio=0.7))
#     plt.rcParams.update({'font.size': 12,"axes.titlesize": 12,"axes.labelsize": 12,"xtick.labelsize" : 12,"ytick.labelsize": 12})
#     fig,ax = plt.subplots()

#     if xmask is None:
#         xmask = np.ones(x.shape,dtype=bool)
#     if show_radargram:
#         ax.pcolormesh(x[xmask]-gl_pos,mat["Elevation"].flatten(),mat["TopoData"][:,xmask],cmap="gray")
#     else:
#         ax.fill_between(x[xmask]-gl_pos,mat["SurfaceElevation"].flatten()[xmask],base[xmask],color="black",alpha=0.075,linewidth=0.0)
#         #ax.axvline(x[minind1],linestyle='dotted',color=color_opts["colors"]["boundary_condition"])
#         #ax.axvline(x[minind],linestyle='dotted',color=color_opts["colors"]["boundary_condition"])
#         #ax.axvline(80,linestyle='dotted',color=color_opts["colors"]["boundary_condition"])

#     if box:
#         rect = mpl.patches.Rectangle((xlim_base[0]-gl_pos, ylim_base[0]), xlim_base[1]-xlim_base[0], ylim_base[1]-ylim_base[0],
#                                         linestyle="solid", linewidth=1.5, edgecolor=color_opts["colors"]["boundary_condition"], facecolor="none",
#                                         zorder=10)
#         ax.add_patch(rect)
#     else:
#         ax.set_xticks(np.linspace(x[xmask][0]-gl_pos,x[xmask][-1]-gl_pos,5))
#     for kk in range(0,nl):
#         ax.plot(x[xmask]-gl_pos,mat["layers_elevation"][kk,xmask],color = color_opts["colors"]["observation"],linewidth=1.0)
#         if label_layers:
#             ax.text(x[xmask][kk*10]-gl_pos,mat["layers_elevation"][kk,xmask][kk*10],str(kk+1),fontsize=15,color="black",
#                     horizontalalignment='left',verticalalignment='center')
#     ax.plot(x[xmask]-gl_pos,base[xmask],"black",linewidth=1.0)
#     ax.plot(x[xmask]-gl_pos,mat["BedmachineBed"].flatten()[xmask],linewidth=1.0,color="black")
#     ax.plot(x[xmask]-gl_pos,mat["SurfaceElevation"].flatten()[xmask],linewidth=1.0,color="black")




#     coords = np.vstack((np.concatenate((x[xmask]-gl_pos, x[xmask][::-1]-gl_pos)), np.concatenate((mat["SurfaceElevation"].flatten()[xmask]*0+RefHeight, mat["SurfaceElevation"].flatten()[xmask][::-1]))))
#     patch1 = mpl.patches.Polygon(xy=coords.T,color=np.array([136, 233, 253])/255) #çolor in sky
#     coords = np.vstack((np.concatenate((x[xmask]-gl_pos, x[xmask][::-1]-gl_pos)), np.concatenate((base[xmask], base[xmask][::-1]*0 + MinElevation))))
#     patch2 = mpl.patches.Polygon(xy=coords.T,color=np.array([119, 158, 185])/255) #color in ocean
#     coords = np.vstack((np.concatenate((x[xmask]-gl_pos, x[xmask][::-1]-gl_pos)), np.concatenate((mat["BedmachineBed"].flatten()[xmask], mat["BedmachineBed"].flatten()[xmask][::-1]*0 + MinElevation)))) #color in ground
#     patch3 = mpl.patches.Polygon(xy=coords.T,color=np.array([168, 104, 66])/255)
#     ax.add_patch(patch1)
#     ax.add_patch(patch2)
#     ax.add_patch(patch3)
#     if xlim is not None:
#         ax.set_xlim(xlim)
#     if ylim is not None:
#         ax.set_ylim(ylim)

#     ax.set_xlabel("Distance from GL [km]")
#     ax.set_ylabel("Elevation [m.a.s.l.]")
#     ax.spines['bottom'].set_bounds(x[xmask][0]-gl_pos+0.005,x[xmask][-1]-gl_pos)
#     ax.xaxis.set_major_formatter(FormatStrFormatter('%.1f'))

#     return fig,ax




In [ ]:

fig,ax = radarplot(distance,xmask=shelf_mask)
fig_name = Path(output_dir,"paper_figures","Radar","radar_whole_shelf.pdf")
fig_name.parent.mkdir(parents=True,exist_ok=True)
# fig.savefig(fig_name)


In [ ]:
fig,ax = radarplot(distance,show_radargram=True,box=False,xmask=shelf_mask*mask,ylim=ylim_base,label_layers=True)
fig_name = Path(output_dir,"paper_figures","Radar","zoomed_shelf_radargram.png")
fig_name.parent.mkdir(parents=True,exist_ok=True)
fig.savefig(fig_name)


In [ ]:
help(plt.pcolormesh)

In [ ]:

twt_max = np.max(mat["twt"])
fig,ax = plt.subplots()
ax.pcolormesh(distances[:,shelf_mask*mask],elevations[:,shelf_mask*mask],mat["data"][1:,shelf_mask*mask],cmap="gray")
# for kk in range(0,nl):
#     ax.plot(distance[shelf_mask*mask],mat["twt_layers"][kk,shelf_mask*mask],color = color_opts["colors"]["observation"],linewidth=1.0)
for kk in range(0,nl):
    ax.plot(distance[shelf_mask*mask],mat["elev_layers"][kk,shelf_mask*mask],color = color_opts["colors"]["observation"],linewidth=1.0)

ax.set_ylim([-100,60])
# flip the y axis
# ax.invert_yaxis()